In [1]:
#pip install nextrec

import logging
import sys

logger = logging.getLogger() 
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter('%(asctime)s %(levelname)s %(message)s'))
logger.handlers = [handler] 

# 5-Minute Quick Start

该Notebook将为大家了解 NextRec：一个统一、高效、可扩展的推荐系统框架，并带大家从零到一训练并构建一个可上线的推荐模型。示例数据集来自电商场景。

开始以前，请通过命令行安装nextrec：

```bash
# 正式版
pip install nextrec

# 测试版
pip install -i https://test.pypi.org/simple/ nextrec
```

先介绍一些推荐系统的概念。在推荐系统中，通常会处理多种类型的输入信号，在经过一系列的变换之后转化为向量输入网络：

- 稠密特征（数值型）：连续或可序数化的数值，如年龄、价格、时长、打分；常见做法是标准化/归一化或对数变换。
- 稀疏特征（类别/ID）：高基数离散字段，如用户 ID、物品 ID、性别、职业、设备类型；通常需要索引化后，在一个embedding lookup matrix中进行嵌入。
- 序列特征（行为序列）：可变长的历史行为，如用户的浏览/点击/购买列表。这类特征表征了用户的行为和兴趣变化，通常我们需要截断、padding，嵌入后通过不同聚合方式（如 mean/sum/attention）将其变为定长向量。
- 上下文特征：时间、地理、曝光位置等环境信息，可是稠密也可能是稀疏，常与主特征交互。
- 多模态特征：文本、图片、视频等经过预训练模型得到的向量，可直接作为稠密输入，或与 ID 交互建模。

通常一个标准的训练数据格式如下所示：

```text
user_id,item_id,gender,age,occupation,history_seq,label
1024,501,1,28,3,"[12,45,18,77]",1
2048,777,0,35,5,"[8,99]",0
```

这里，我们提供了一份脱敏后的数据集供大家使用，该数据集来源于电商场景下，包含用户id，物品id，稠密特征，稀疏特征和序列特征，标签则包含是否点击，与是否转化。

In [13]:
pip show nextrec

Name: nextrec
Version: 0.4.21
Summary: A comprehensive recommendation library with match, ranking, and multi-task learning models
Home-page: https://github.com/zerolovesea/NextRec
Author: Yang Zhou
Author-email: zyaztec@gmail.com
License: 
Location: /opt/anaconda3/envs/nextrec/lib/python3.10/site-packages
Editable project location: /Users/zyaztec/DailyWork/建模代码整理/NextRec
Requires: numpy, pandas, pyarrow, pyyaml, rich, scikit-learn, scipy, swanlab, torch, torchvision, transformers, wandb
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from nextrec.data.preprocessor import DataProcessor

df = pd.read_csv('/NextRec/dataset/multitask_task.csv')
df.head()

,user_id,item_id,dense_0,dense_1,dense_2,dense_3,dense_4,dense_5,dense_6,dense_7,...,sparse_5,sparse_6,sparse_7,sparse_8,sparse_9,sparse_10,sparse_11,sequence_0,click,conversion
0,1,7817,0.147041,0.310204,0.777809,0.944897,0.623154,0.571242,0.770095,0.321103,...,161,138,88,5,312,416,188,"[90, 54, 86, 5, 121, 138, 45, 100, 0, 0, 0, 0,...",1,0
1,1,3579,0.778112,0.803593,0.518520,0.910912,0.043562,0.821427,0.880369,0.337482,...,252,25,402,7,168,155,154,"[3, 95, 31, 124, 56, 79, 109, 0, 0, 0, 0, 0, 0...",1,1
2,1,2657,0.586647,0.123208,0.203636,0.116398,0.240645,0.882588,0.062836,0.629869,...,27,62,145,109,432,170,133,"[139, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,0
3,1,2689,0.337401,0.705511,0.138758,0.945233,0.330333,0.377462,0.121577,0.427124,...,241,144,40,6,333,175,210,"[59, 29, 34, 106, 4, 103, 0, 0, 0, 0, 0, 0, 0,...",1,0
4,1,2495,0.669473,0.564266,0.006319,0.255851,0.698055,0.052065,0.583597,0.590456,...,152,27,204,129,319,97,168,"[52, 122, 104, 116, 5, 138, 37, 30, 59, 10, 19...",1,0


In [4]:
task_labels = ['click', 'conversion']
dense_features_list = [col for col in df.columns if 'dense' in col]
sparse_features_list = [col for col in df.columns if 'sparse' in col] + ['user_id', 'item_id']
sequence_features_list = [col for col in df.columns if 'sequence' in col]

# 由于csv中的序列特征是以字符串形式存储的列表，我们需要先将其转换为真正的列表对象
for col in df.columns:
    if 'sequence' in col:
        df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)

将数据处理成需要的格式以后，我们将拆分出训练集和推理集，用来给模型进行指标评估。

In [5]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(df, test_size=0.2, random_state=2025)

现在我们开始准备模型，我们需要将模型需要的不同特征进行定义，并传给模型，这里需要用到nextrec内置的三种特征DenseFeature, SequenceFeature, SparseFeature。

In [ ]:
from nextrec.basic.features import DenseFeature, SequenceFeature, SparseFeature

# 我们将所有的数值特征作为DenseFeature，proj_dim=1表示不进行投影，当proj_dim大于1时表示对数值特征进行线性变换，类似于embedding的效果
dense_features = [DenseFeature(name=feat, proj_dim=1) for feat in dense_features_list] 

# 稀疏特征和序列特征我们一般会进行embedding，embedding_dim可以根据实际情况调整
# vocab_size 需要覆盖数据里的最大取值（从0开始），避免 embedding 索引越界
sparse_features = []
for feat in sparse_features_list:
    vocab_size = 20001
    # sparsefeature还可以设置一些其他的参数，例如初始化器，正则化和embedding_name等，当两个特征共享embedding时，可以设置相同的embedding_name       
    sparse_features.append(SparseFeature(name=feat, vocab_size=vocab_size, embedding_dim=4, embedding_name=feat)) 

# 序列特征的处理和稀疏特征类似，不过还需要设置序列的最大长度max_len和padding_idx等参数
sequence_features = []
for feat in sequence_features_list:
    vocab_size = 500
    sequence_features.append(
        SequenceFeature(
            name=feat,
            vocab_size=vocab_size,
            max_len=20,
            embedding_dim=8,
            padding_idx=0
        )
    )


现在，我们要用到DataLoader了，DataLoader通常为模型准备不断迭代的批次的数据。别担心，我们为你准备了RecDataLoader，它用起来不会很复杂。

RecDataLoader是一个强大的工具，它支持传入dict，DataFrame，DataLoader以及一个路径，并且支持配置流式加载数据，只要选择streaming=True。这个实例适配了NextRec框架下的一切训练场景，为了避免不必要的困扰，我们强烈建议你试着用它。

当然你同样可以不用它，NextRec也支持使用dict，DataFrame直接进行训练，稍后你就能看到。

In [7]:
from nextrec.data.dataloader import RecDataLoader

task_labels = ['click', 'conversion']

dataloader = RecDataLoader(
    dense_features=dense_features,
    sparse_features=sparse_features,
    sequence_features=sequence_features,
    target=task_labels,
)

# 我们需要为训练集和验证集分别创建dataloader
train_loader = dataloader.create_dataloader(
    data=train_df,
    batch_size=512,
    shuffle=True,
)

valid_loader = dataloader.create_dataloader(
    data=valid_df,
    batch_size=512,
    shuffle=False,
)


# 你也可以传入一个路径来配置流式数据加载器
# train_loader = dataloader.create_dataloader(
#     data='/path/to/train/data',
#     batch_size=512,
#     shuffle=True,
#     streaming=True
# )

终于来到这一步了，我们来选择想要训练的模型。NextRec提供了超过20种工业界常用的召回，精排，多任务模型。这里我们先用经典的MMOE训练一个模型。在此之前，我们需要实例化模型，并且为模型分配参数。

在实例化模型以后，我们需要编译模型，为训练器分配优化器，调度器和损失函数。NextRec支持超过8种优化器，10种调度器和20种损失函数以及不平衡损失。

In [8]:
from nextrec.models.multi_task.mmoe import MMOE

# mmoe需要设置专家网络和任务塔的参数，这里我们设置4个专家网络，每个专家网络包含两层，任务塔也包含两层
# 我们拥有两个任务，分别是click和conversion，每个任务都是二分类任务，因此task参数设置为['binary', 'binary']
model = MMOE(
    dense_features=dense_features,
    sparse_features=sparse_features,
    sequence_features=sequence_features,
    expert_params= {"dims": [128, 64],  "activation": "leaky_relu", "dropout": 0.3},
    num_experts=4,  # 4 expert networks
    tower_params_list=[{"dims": [64, 32], "activation": "leaky_relu", "dropout": 0.2},  # click task
                       {"dims": [64, 32], "activation": "leaky_relu", "dropout": 0.2},  # conversion task
                        ],
    target=task_labels,  # multiple task labels
    task=['binary', 'binary'],  # each task type
    device='cpu',
    embedding_l1_reg=1e-6,
    embedding_l2_reg=1e-5,
    dense_l1_reg=1e-5,
    dense_l2_reg=1e-4,
    session_id="mmoe_task"    # session id用于区分不同的训练任务，会将训练日志，checkpoint，模型参数等保存在以session_id命名的文件夹中
)

# 编译模型用于设置优化器和损失函数，统一在 compile 中配置。
# 这里我们使用adam优化器，学习率为1e-3，权重衰减为1e-5
# 每个任务的损失函数我们都使用二分类交叉熵损失函数
model.compile(
    optimizer="adam",
    optimizer_params={"lr": 1e-3, "weight_decay": 1e-5},
    loss=['bce', 'bce'],  # loss for each task
    # loss_weights="grad_norm" # 使用grad_norm自动调整每个任务的损失权重，你也可以传入一个list来手动设置每个任务的损失权重，例如loss_weights=[1.0, 0.5]
)

# 现在我们可以开始训练模型了，这里我们设置训练3个epoch，你可以根据实际情况调整
# 同时我们还可以为每个任务设置评估指标，这里我们为每个任务都设置了AUC, Recall和Precision指标
# 注意你可以在nextrec_logs/mmoe_iflytek文件夹中查看训练日志和模型checkpoint
# 通过设置use_tensorboard，use_wandb，use_swanlab参数以及wandb_kwargs，swanlab_kwargs，可以将训练日志同步到对应的平台上。
# 例如：use_swanlab=True, swanlab_kwargs={"project": "NextRec", "name": "MMOE_experiment"}
model.fit(
    train_data=train_loader, 
    valid_data=valid_loader,
    metrics={
        'click': ['auc', 'recall', 'precision'],
        'conversion': ['auc', 'recall', 'precision']
    },
    epochs=1,
)


Model Summary: MMOE


Feature Configuration
--------------------------------------------------------------------------------
Dense Features (8):
  1. dense_0             
  2. dense_1             
  3. dense_2             
  4. dense_3             
  5. dense_4             
  6. dense_5             
  7. dense_6             
  8. dense_7             

Sparse Features (14):
  #    Name           Vocab Size        Embed Name  Embed Dim
  ---- ------------ ------------ ----------------- ----------
  1    sparse_0              426          sparse_0          4
  2    sparse_1              261          sparse_1          4
  3    sparse_2              453          sparse_2          4
  4    sparse_3              233          sparse_3          4
  5    sparse_4              401          sparse_4          4
  6    sparse_5              312          sparse_5          4
  7    sparse_6              149          sparse_6          4
  8    sparse_7              434          sparse_7          4
  9

Epoch 1: 157/157 elapsed=0:00:04 speed=37.68/s ETA=0:00:00


      Epoch 1/1 - Train (loss=1.1099)       
╭────────────┬────────┬────────┬───────────╮
│ Task       │    auc │ recall │ precision │
├────────────┼────────┼────────┼───────────┤
│ click      │ 0.7993 │ 0.8428 │    0.7907 │
│ conversion │ 0.6656 │ 0.2418 │    0.5594 │
╰────────────┴────────┴────────┴───────────╯

      Epoch 1/1 - Train (loss=1.1099)       
╭────────────┬────────┬────────┬───────────╮
│ Task       │    auc │ recall │ precision │
├────────────┼────────┼────────┼───────────┤
│ click      │ 0.7993 │ 0.8428 │    0.7907 │
│ conversion │ 0.6656 │ 0.2418 │    0.5594 │
╰────────────┴────────┴────────┴───────────╯

             Epoch 1/1 - Valid              
╭────────────┬────────┬────────┬───────────╮
│ Task       │    auc │ recall │ precision │
├────────────┼────────┼────────┼───────────┤
│ click      │ 0.8562 │ 0.8530 │    0.8258 │
│ conversion │ 0.7185 │ 0.4963 │    0.5560 │
╰────────────┴────────┴────────┴───────────╯

             Epoch 1/1 - Valid              
╭────────────┬────────┬────────┬───────────╮
│ Task       │    auc │ recall │ precision │
├────────────┼────────┼────────┼───────────┤
│ click      │ 0.8562 │ 0.8530 │    0.8258 │
│ conversion │ 0.7185 │ 0.4963 │    0.5560 │
╰────────────┴────────┴────────┴───────────╯

Saved checkpoint to /Users/zyaztec/DailyWork/建模代码整理/NextRec/tutorials/notebooks/zh/nextrec_logs/mmoe_task/MMOE_checkpoint.pt
Saved checkpoint to /Users/zyaztec/DailyWork/建模代码整理/NextRec/tutorials/notebooks/zh/nextrec_logs/mmoe_task/MMOE_best.pt
Saved best model to:               /Users/zyaztec/DailyWork/建模代码整理/NextRec/tutorials/notebooks/zh/nextrec_logs/mmoe_task/MMOE_best.pt with val_auc_click: 0.856220
Restoring model weights from epoch: 1 with best val_auc_click: 0.856220

Training finished.

Load best model from:              /Users/zyaztec/DailyWork/建模代码整理/NextRec/tutorials/notebooks/zh/nextrec_logs/mmoe_task/MMOE_best.pt


MMOE(
  (embedding): EmbeddingLayer(
    (embed_dict): ModuleDict(
      (sparse_0): Embedding(426, 4, padding_idx=0)
      (sparse_1): Embedding(261, 4, padding_idx=0)
      (sparse_2): Embedding(453, 4, padding_idx=0)
      (sparse_3): Embedding(233, 4, padding_idx=0)
      (sparse_4): Embedding(401, 4, padding_idx=0)
      (sparse_5): Embedding(312, 4, padding_idx=0)
      (sparse_6): Embedding(149, 4, padding_idx=0)
      (sparse_7): Embedding(434, 4, padding_idx=0)
      (sparse_8): Embedding(148, 4, padding_idx=0)
      (sparse_9): Embedding(489, 4, padding_idx=0)
      (sparse_10): Embedding(429, 4, padding_idx=0)
      (sparse_11): Embedding(233, 4, padding_idx=0)
      (user_id): Embedding(5001, 4, padding_idx=0)
      (item_id): Embedding(10001, 4, padding_idx=0)
      (sequence_0): Embedding(155, 8, padding_idx=0)
    )
    (dense_transforms): ModuleDict()
    (sequence_poolings): ModuleDict(
      (sequence_0): AveragePooling()
    )
  )
  (experts): ModuleList(
    (0-3): 

现在我们再来训练一个精排模型，这里我们训练一个AutoINT作为示例，任务目标从多目标变为单目标。这是一篇北京大学发表在 CIKM 2019 的文章，模型的相关讲解可以在[这里](https://guyuecanhui.github.io/2020/05/09/paper-2019-pku-autoint/)看到。

In [10]:
from nextrec.models.ranking.autoint import AutoInt

target = 'conversion'

# 由于目标变了，我们重新创建dataloader
dataloader = RecDataLoader(
    dense_features=dense_features,
    sparse_features=sparse_features,
    sequence_features=sequence_features,
    target=target,
)

train_loader = dataloader.create_dataloader(
    data=train_df,
    batch_size=512,
    shuffle=True,
)

valid_loader = dataloader.create_dataloader(
    data=train_df,
    batch_size=512,
    shuffle=False,
)

model = AutoInt(
    dense_features=dense_features,
    sparse_features=sparse_features,
    sequence_features=sequence_features,
    att_layer_num=3,
    att_embedding_dim=8,
    att_head_num=2,
    att_dropout=0.0,
    att_use_residual=True,
    target=target,
    device='cpu',
    embedding_l1_reg=1e-6,
    dense_l1_reg=1e-5,
    embedding_l2_reg=1e-5,
    dense_l2_reg=1e-4,
    session_id="autoint_iflytek"
)

# 编译模型
model.compile(
    optimizer="adam",
    optimizer_params={
        "lr": 1e-3,
        "weight_decay": 1e-5
    },
    loss="bce",
)

# 训练模型
model.fit(
    train_data=train_loader,
    valid_data=valid_loader,
    metrics=['auc',
             'recall',
             'precision'],
    epochs=3,
    batch_size=512,
    shuffle=True,
)


Model Summary: AUTOINT


Feature Configuration
--------------------------------------------------------------------------------
Dense Features (8):
  1. dense_0             
  2. dense_1             
  3. dense_2             
  4. dense_3             
  5. dense_4             
  6. dense_5             
  7. dense_6             
  8. dense_7             

Sparse Features (14):
  #    Name           Vocab Size        Embed Name  Embed Dim
  ---- ------------ ------------ ----------------- ----------
  1    sparse_0              426          sparse_0          4
  2    sparse_1              261          sparse_1          4
  3    sparse_2              453          sparse_2          4
  4    sparse_3              233          sparse_3          4
  5    sparse_4              401          sparse_4          4
  6    sparse_5              312          sparse_5          4
  7    sparse_6              149          sparse_6          4
  8    sparse_7              434          sparse_7          4


Epoch 1: 157/157 elapsed=0:00:04 speed=38.78/s ETA=0:00:00


      Epoch 1/3 - Train (loss=0.6253)       
╭────────────┬────────┬────────┬───────────╮
│ Task       │    auc │ recall │ precision │
├────────────┼────────┼────────┼───────────┤
│ conversion │ 0.5959 │ 0.0925 │    0.5539 │
╰────────────┴────────┴────────┴───────────╯

      Epoch 1/3 - Train (loss=0.6253)       
╭────────────┬────────┬────────┬───────────╮
│ Task       │    auc │ recall │ precision │
├────────────┼────────┼────────┼───────────┤
│ conversion │ 0.5959 │ 0.0925 │    0.5539 │
╰────────────┴────────┴────────┴───────────╯

             Epoch 1/3 - Valid              
╭────────────┬────────┬────────┬───────────╮
│ Task       │    auc │ recall │ precision │
├────────────┼────────┼────────┼───────────┤
│ conversion │ 0.6998 │ 0.2901 │    0.5975 │
╰────────────┴────────┴────────┴───────────╯

...

      Epoch 3/3 - Train (loss=0.5168)       
╭────────────┬────────┬────────┬───────────╮
│ Task       │    auc │ recall │ precision │
├────────────┼────────┼────────┼───────────┤
│ conversion │ 0.7862 │ 0.4869 │    0.6610 │
╰────────────┴────────┴────────┴───────────╯

             Epoch 3/3 - Valid              
╭────────────┬────────┬────────┬───────────╮
│ Task       │    auc │ recall │ precision │
├────────────┼────────┼────────┼───────────┤
│ conversion │ 0.8437 │ 0.5907 │    0.7210 │
╰────────────┴────────┴────────┴───────────╯

             Epoch 3/3 - Valid              
╭────────────┬────────┬────────┬───────────╮
│ Task       │    auc │ recall │ precision │
├────────────┼────────┼────────┼───────────┤
│ conversion │ 0.8437 │ 0.5907 │    0.7210 │
╰────────────┴────────┴────────┴───────────╯

Saved checkpoint to /Users/zyaztec/DailyWork/建模代码整理/NextRec/tutorials/notebooks/zh/nextrec_logs/autoint_iflytek/AUTOINT_checkpoint.pt
Saved checkpoint to /Users/zyaztec/DailyWork/建模代码整理/NextRec/tutorials/notebooks/zh/nextrec_logs/autoint_iflytek/AUTOINT_best.pt
Saved best model to:               /Users/zyaztec/DailyWork/建模代码整理/NextRec/tutorials/notebooks/zh/nextrec_logs/autoint_iflytek/AUTOINT_best.pt with val_auc: 0.843661
Restoring model weights from epoch: 3 with best val_auc: 0.843661

Training finished.

Load best model from:              /Users/zyaztec/DailyWork/建模代码整理/NextRec/tutorials/notebooks/zh/nextrec_logs/autoint_iflytek/AUTOINT_best.pt


AutoInt(
  (embedding): EmbeddingLayer(
    (embed_dict): ModuleDict(
      (sparse_0): Embedding(426, 4, padding_idx=0)
      (sparse_1): Embedding(261, 4, padding_idx=0)
      (sparse_2): Embedding(453, 4, padding_idx=0)
      (sparse_3): Embedding(233, 4, padding_idx=0)
      (sparse_4): Embedding(401, 4, padding_idx=0)
      (sparse_5): Embedding(312, 4, padding_idx=0)
      (sparse_6): Embedding(149, 4, padding_idx=0)
      (sparse_7): Embedding(434, 4, padding_idx=0)
      (sparse_8): Embedding(148, 4, padding_idx=0)
      (sparse_9): Embedding(489, 4, padding_idx=0)
      (sparse_10): Embedding(429, 4, padding_idx=0)
      (sparse_11): Embedding(233, 4, padding_idx=0)
      (user_id): Embedding(5001, 4, padding_idx=0)
      (item_id): Embedding(10001, 4, padding_idx=0)
      (sequence_0): Embedding(155, 8, padding_idx=0)
    )
    (dense_transforms): ModuleDict()
    (sequence_poolings): ModuleDict(
      (sequence_0): AveragePooling()
    )
  )
  (projection_layers): ModuleList

觉得手动创建dataloader很麻烦？NextRec同样支持直接传入dataframe或dict -- 只要的你内存够大。（当然，再次强调，RecDataloader是一个更好的选择。）

你也可以不传入valid_data，这样模型就会在全量数据集上训练。

或者你可以设置valid_split参数，来让模型自动从训练集上拆分出验证集。

In [12]:
model.fit(
    train_data=train_df,
    metrics=['auc',
             'recall',
             'precision'],
    epochs=1,
    batch_size=512,
    shuffle=True,
    valid_split=0.2
)

Split data: 64000 training samples, 16000 validation samples

Model Summary: AUTOINT


Feature Configuration
--------------------------------------------------------------------------------
Dense Features (8):
  1. dense_0             
  2. dense_1             
  3. dense_2             
  4. dense_3             
  5. dense_4             
  6. dense_5             
  7. dense_6             
  8. dense_7             

Sparse Features (14):
  #    Name           Vocab Size        Embed Name  Embed Dim
  ---- ------------ ------------ ----------------- ----------
  1    sparse_0              426          sparse_0          4
  2    sparse_1              261          sparse_1          4
  3    sparse_2              453          sparse_2          4
  4    sparse_3              233          sparse_3          4
  5    sparse_4              401          sparse_4          4
  6    sparse_5              312          sparse_5          4
  7    sparse_6              149          sparse_6          4
 

Epoch 1: 125/125 elapsed=0:00:03 speed=40.05/s ETA=0:00:00


      Epoch 1/1 - Train (loss=0.4680)       
╭────────────┬────────┬────────┬───────────╮
│ Task       │    auc │ recall │ precision │
├────────────┼────────┼────────┼───────────┤
│ conversion │ 0.8330 │ 0.5777 │    0.7036 │
╰────────────┴────────┴────────┴───────────╯

      Epoch 1/1 - Train (loss=0.4680)       
╭────────────┬────────┬────────┬───────────╮
│ Task       │    auc │ recall │ precision │
├────────────┼────────┼────────┼───────────┤
│ conversion │ 0.8330 │ 0.5777 │    0.7036 │
╰────────────┴────────┴────────┴───────────╯

             Epoch 1/1 - Valid              
╭────────────┬────────┬────────┬───────────╮
│ Task       │    auc │ recall │ precision │
├────────────┼────────┼────────┼───────────┤
│ conversion │ 0.8088 │ 0.5348 │    0.6714 │
╰────────────┴────────┴────────┴───────────╯

             Epoch 1/1 - Valid              
╭────────────┬────────┬────────┬───────────╮
│ Task       │    auc │ recall │ precision │
├────────────┼────────┼────────┼───────────┤
│ conversion │ 0.8088 │ 0.5348 │    0.6714 │
╰────────────┴────────┴────────┴───────────╯

Saved checkpoint to /Users/zyaztec/DailyWork/建模代码整理/NextRec/tutorials/notebooks/zh/nextrec_logs/autoint_iflytek/AUTOINT_checkpoint.pt
Saved checkpoint to /Users/zyaztec/DailyWork/建模代码整理/NextRec/tutorials/notebooks/zh/nextrec_logs/autoint_iflytek/AUTOINT_best.pt
Saved best model to:               /Users/zyaztec/DailyWork/建模代码整理/NextRec/tutorials/notebooks/zh/nextrec_logs/autoint_iflytek/AUTOINT_best.pt with val_auc: 0.808796
Restoring model weights from epoch: 1 with best val_auc: 0.808796

Training finished.

Load best model from:              /Users/zyaztec/DailyWork/建模代码整理/NextRec/tutorials/notebooks/zh/nextrec_logs/autoint_iflytek/AUTOINT_best.pt


AutoInt(
  (embedding): EmbeddingLayer(
    (embed_dict): ModuleDict(
      (sparse_0): Embedding(426, 4, padding_idx=0)
      (sparse_1): Embedding(261, 4, padding_idx=0)
      (sparse_2): Embedding(453, 4, padding_idx=0)
      (sparse_3): Embedding(233, 4, padding_idx=0)
      (sparse_4): Embedding(401, 4, padding_idx=0)
      (sparse_5): Embedding(312, 4, padding_idx=0)
      (sparse_6): Embedding(149, 4, padding_idx=0)
      (sparse_7): Embedding(434, 4, padding_idx=0)
      (sparse_8): Embedding(148, 4, padding_idx=0)
      (sparse_9): Embedding(489, 4, padding_idx=0)
      (sparse_10): Embedding(429, 4, padding_idx=0)
      (sparse_11): Embedding(233, 4, padding_idx=0)
      (user_id): Embedding(5001, 4, padding_idx=0)
      (item_id): Embedding(10001, 4, padding_idx=0)
      (sequence_0): Embedding(155, 8, padding_idx=0)
    )
    (dense_transforms): ModuleDict()
    (sequence_poolings): ModuleDict(
      (sequence_0): AveragePooling()
    )
  )
  (projection_layers): ModuleList

下面是已经支持的模型，欢迎调用来测试效果

### 排序模型

| 模型 | 论文 | 年份 | 状态 |
|------|------|------|------|
| **FM** | Factorization Machines | ICDM 2010 | 已支持 |
| **AFM** | Attentional Factorization Machines: Learning the Weight of Feature Interactions via Attention Networks | IJCAI 2017 | 已支持 |
| **DeepFM** | DeepFM: A Factorization-Machine based Neural Network for CTR Prediction | IJCAI 2017 | 已支持 |
| **Wide&Deep** | Wide & Deep Learning for Recommender Systems | DLRS 2016 | 已支持 |
| **xDeepFM** | xDeepFM: Combining Explicit and Implicit Feature Interactions | KDD 2018 | 已支持 |
| **FiBiNET** | FiBiNET: Combining Feature Importance and Bilinear Feature Interaction for CTR Prediction | RecSys 2019 | 已支持 |
| **PNN** | Product-based Neural Networks for User Response Prediction | ICDM 2016 | 已支持 |
| **AutoInt** | AutoInt: Automatic Feature Interaction Learning | CIKM 2019 | 已支持 |
| **DCN** | Deep & Cross Network for Ad Click Predictions | ADKDD 2017 | 已支持 |
| **DIN** | Deep Interest Network for Click-Through Rate Prediction | KDD 2018 | 已支持 |
| **DIEN** | Deep Interest Evolution Network for Click-Through Rate Prediction | AAAI 2019 | 已支持 |
| **MaskNet** | MaskNet: Introducing Feature-wise Gating Blocks for High-dimensional Sparse Recommendation Data | 2020 | 已支持 |

### 召回模型

| 模型 | 论文 | 年份 | 状态 |
|------|------|------|------|
| **DSSM** | Learning Deep Structured Semantic Models | CIKM 2013 | 已支持 |
| **DSSM v2** | DSSM with pairwise BPR-style optimization | - | 已支持 |
| **YouTube DNN** | Deep Neural Networks for YouTube Recommendations | RecSys 2016 | 已支持 |
| **MIND** | Multi-Interest Network with Dynamic Routing | CIKM 2019 | 已支持 |
| **SDM** | Sequential Deep Matching Model | - | 已支持 |

### 多任务模型

| 模型 | 论文 | 年份 | 状态 |
|------|------|------|------|
| **MMOE** | Modeling Task Relationships in Multi-task Learning | KDD 2018 | 已支持 |
| **PLE** | Progressive Layered Extraction | RecSys 2020 | 已支持 |
| **ESMM** | Entire Space Multi-Task Model | SIGIR 2018 | 已支持 |
| **ShareBottom** | Multitask Learning | - | 已支持 |